In [1]:
from tqdm.notebook import tqdm
from IPython.display import display, HTML

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import ConvLSTM2D, Conv2D, Conv3D, Dense, TimeDistributed, MaxPooling2D, GlobalAveragePooling2D 
from tensorflow.keras.layers import Activation, Dropout, Flatten, BatchNormalization, Input, LSTM

2024-05-17 23:23:20.283502: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 23:23:21.426282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat_multi')
df = source.read()

In [3]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [4]:
target = source.metadata['categories']['multi'] # multi / trinary

labels = df[target].to_numpy()

In [5]:
class KerasModelCreator:
    def crop_y(self, y):
        lower = 0.1
        bounded_y = np.where(y < lower, 0.0, y)
        rescaled_y = bounded_y/bounded_y.sum(axis=1, keepdims=1)
        return rescaled_y
        
    def normalise_X(self, X, p=1):
        upper = np.percentile(X, 100-p)
        lower = np.percentile(X, p)
    
        bounded_X = np.where(X > upper, np.median(X), X)
        bounded_X = np.where(X < lower, np.median(X), bounded_X)
        
        scaled_X = (bounded_X - lower)/(upper - lower)
        return scaled_X
        
    def split_and_normalise(self, y, X, random_state=42, test_size=0.1):
        """Split and max scale."""
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, shuffle=False)

        y_train, y_test = self.crop_y(y_train), self.crop_y(y_test)

        for i in range(X_train.shape[-1]):
            X_train[...,i] = self.normalise_X(X_train[...,i])
            X_test[...,i] = self.normalise_X(X_test[...,i])
        
        return X_train, X_test, y_train, y_test

    def build_model(self, y_train, input_size, metrics):
        filters_scale = 128
        dropout_base = 0.25
        
        m = tf.keras.Sequential()

        m.add(Input(input_size))

        m.add(Conv2D(
            filters=filters_scale*2, kernel_size=5, 
            padding='same', activation='relu'
        ))
        m.add(MaxPooling2D())

        m.add(Conv2D(
            filters=filters_scale, kernel_size=5, 
            padding='same', activation='relu'
        ))
        m.add(MaxPooling2D())

        m.add(GlobalAveragePooling2D())

        fm = tf.keras.Sequential()
        fm.add(Input((None, *input_size)))
        fm.add(TimeDistributed(m))

        fm.add(LSTM(filters_scale))

        fm.add(Dropout(dropout_base*2))
        
        fm.add(Dense(filters_scale*2, activation="relu"))
        fm.add(Dropout(dropout_base))

        fm.add(Dense(filters_scale, activation="relu"))
        fm.add(Dropout(dropout_base))
        
        fm.add(Dense(
            y_train.shape[1], 
            activation='softmax', 
        ))

        loss = tf.keras.losses.Huber(
            delta=1.0,
            reduction='sum_over_batch_size'
        )

        opt = tf.keras.optimizers.Adam(
            learning_rate=0.001
        )
        fm.compile(
            optimizer=opt,
            loss=loss,
            metrics=metrics
        )
        return fm

        
    def run(self, y, X, model_path, epochs=10, overwrite=False):
        ''' 
        If not overwrite and there's an existing model, the model will 
        continue training if the given epoch is bigger than the previous,
        else just evaluate.
        Ensure train splits are the same across continuations / evaluations
        by not modifying the random_state in split_and_normalise.
        '''
        model_savepoint = model_path.parent.joinpath(model_path.stem)
        log_file = model_path.with_suffix('.log')

        if overwrite:
            for f in [model_path, log_file] + list(model_savepoint.glob('*')):
                f.unlink(missing_ok=True)

        X_train, X_test, y_train, y_test = self.split_and_normalise(y, X, random_state=42)
        
        default_metrics = ['accuracy', 'root_mean_squared_error', 'mean_squared_error', 'r2_score']
        
        if model_path.is_file():
            model = tf.keras.models.load_model(model_path)
        else:
            model = self.build_model(y_train, X_train.shape, default_metrics)

        callbacks = [
            tf.keras.callbacks.BackupAndRestore(
                model_savepoint, save_freq='epoch', delete_checkpoint=False
            ),
            tf.keras.callbacks.CSVLogger(
                log_file, append=True
            )
        ]

        if log_file.is_file():
            df = pd.read_csv(log_file)[['epoch', 'loss'] + default_metrics]
            df['epoch'] += 1
            print('Previous training:')
                
            display(HTML(df.to_html(index=False)))
        
        model.fit(
            X_train, y_train, epochs=epochs, verbose=1, batch_size=1, callbacks=callbacks, shuffle=False
        )

        model.save(model_path)
        
        return model.evaluate(X_test, y_test, verbose=0, return_dict=True)
    

In [ ]:
model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

shuffle_seed = 42
shuffled_labels = shuffle(labels, random_state=shuffle_seed)

test_years = [2020]

train_years = [2017, 2018, 2019]
all_seasons = [3, 6, 9, 12]

In [ ]:
# tf.keras.utils.get_custom_objects().clear()
# @tf.keras.utils.register_keras_serializable(name='f1_majority')
# def f1_majority(y_true, y_pred):
#     # Convert multilabel proportions to single-label majority class
#     majority_class = (y_true == tf.keras.ops.amax(y_true, keepdims=True, axis=1))
#     majority_class = tf.cast(majority_class, tf.float32)
    
#     true_positives = K.sum(K.round(majority_class * y_pred))
#     possible_positives = K.sum(K.round(majority_class))

#     recall = true_positives / (possible_positives + K.epsilon())
    
#     predicted_positives = K.sum(K.round(y_pred))

#     precision = true_positives / (predicted_positives + K.epsilon())
    
#     return 2 * precision * recall / (precision + recall + K.epsilon())